# tf.functiom and auto-graph


In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#tf.functiom and auto-graph
def scaled_elu(z,scale =1.0,alpha = 1.0):
    #z >= 0? scale * z :scale *alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))
print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3.,-4.5])))


tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.988891  ], shape=(2,), dtype=float32)


In [3]:
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print(scaled_elu_tf(tf.constant([-3.,-4.5])))

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.988891  ], shape=(2,), dtype=float32)


In [4]:
print(scaled_elu_tf.python_function is scaled_elu) 

True


In [5]:
%timeit scaled_elu_tf(tf.random.normal((1000,1000)))
%timeit scaled_elu(tf.random.normal((1000,1000)))
#可以看出tf图函数比普通python函数要快，GPU 加速会更明显

107 ms ± 4.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
112 ms ± 958 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
#1+1/2+1/2^2+...+1/2^n

@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(20))
    

tf.Tensor(1.9999981, shape=(), dtype=float32)


In [7]:
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display,Markdown
    display(Markdown('```pyhton\n{}\n```'.format(code)))

In [8]:
display_tf_code(scaled_elu)

```pyhton
def tf__scaled_elu(z, scale=None, alpha=None):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('scaled_elu', 'scaled_elu_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as scaled_elu_scope:
    is_positive = ag__.converted_call(tf.greater_equal, scaled_elu_scope.callopts, (z, 0.0), None, scaled_elu_scope)
    do_return = True
    retval_ = scaled_elu_scope.mark_return_value(scale * ag__.converted_call(tf.where, scaled_elu_scope.callopts, (is_positive, z, alpha * ag__.converted_call(tf.nn.elu, scaled_elu_scope.callopts, (z,), None, scaled_elu_scope)), None, scaled_elu_scope))
  do_return,
  return ag__.retval(retval_)

```

In [9]:
def display_tf_code(func):
    code = tf.autograph.to_graph(func)
    from IPython.display import display,Markdown
    display(Markdown('```pyhton\n{}\n```'.format(code)))

In [10]:
var = tf.Variable(0.1)#定义一个变量，变量不可以放在图函数里面
#在函数转function之前，要将所有变量全部都初始化

@tf.function
def add_43():
#     var = tf.Variable(0.1)#变量不可以放在图函数里面
    return var.assign_add(43) #+
print(add_43())

tf.Tensor(43.1, shape=(), dtype=float32)


In [11]:
# tf.function(
#     func=None,
#     input_signature=None,
#     autograph=True,
#     experimental_autograph_options=None,
#     experimental_relax_shapes=False,
# )
# function参数:input_signature制定输入类型
@tf.function(input_signature=[tf.TensorSpec([None],tf.int32,name='x')])
def cube(z):
    return tf.pow(z,3)#x^3
try:
    print(cube(tf.constant([1.,2.,3.])))
except ValueError as ex:
    print('ValueError:\n',ex)
print(cube(tf.constant([1,2,3])))

ValueError:
 Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [12]:
#tf.function  py_func-->tf_graph
# get_concrete_function  ->add input_signature->SaveModel

cube_func_int32 = cube.get_concrete_function(
        tf.TensorSpec([None], tf.int32))
print(cube_func_int32)

In [13]:
print(cube_func_int32 is cube.get_concrete_function(
            tf.TensorSpec([5],tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(
            tf.constant([1,2,3])))

True
True


In [14]:
# cube_func_int32有了一些图的方法
cube_func_int32.graph#查看图

In [15]:
cube_func_int32.graph.get_operations()#打印他的几个操作，有四个

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [16]:
pow_op = cube_func_int32.graph.get_operations()[2]
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [17]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [18]:
cube_func_int32.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [19]:
cube_func_int32.graph.get_tensor_by_name("x:0")

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [20]:
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 119
}